# Run application

In this page I will introduce how to run `fastapi` applications in general and in particular how to run examples related to `fastapi` in the site.

<a href="https://fastapi.tiangolo.com/deployment/docker/">Here</a> you can find out more details about using `fastapi` in the docker.

## dockerfile{#sec-dockerfile}

In the next cell is the docker file I am using for this example.

In [9]:
# %load run_application_files/dockerfile
FROM python:3.11
COPY requrements.txt requrements.txt
RUN pip3 install -r requrements.txt
EXPOSE 8000

## requrements.txt

Python libraries you only needed to run the `fastapi` server. It is supposed to be used in the [dockerfile](#sec-dockerfile) described above.

In [6]:
# %load run_application_files/requrements.txt
fastapi==0.103.1
uvicorn==0.23.2

## Program{sec-prog}

You need to declare an object of class `fastapi.fastAPI`. Then use its decorators to add to your functions the ability to respond to certain requests.

So in the following example, I create `fastapi.fastAPI` under the name `my_first_app`, and create a function that will always respond `hello` to a `get` request.

In [10]:
# %load run_application_files/get_started.py
from fastapi import FastAPI

my_first_app = FastAPI()

@my_first_app.get("/")
def say_hello():
    return "hello"

## Run

To run the `fastapi` official documentation recomend to use `uvicorn` as web-server. So you need to use command:

`uvicon --host <host> <path to python file with program>:<name of fastapi.fastAPI object in your program>.`

So in the following example, a docker container is run, tested and stopped:

- Created from the image named `fastapi_experiment` described in the [docker file](#sec-dockerfile) above;
- With the name `test_container` for container;
- The default port for `fastapi` is 8000, so port 8000 on the container is connected to port 8000 on the local machine;
- With volume that allows to read [program](#sec-prog);
- And with the command created from the required pattern.

In [1]:
!docker run --rm -itd\
    --name test_container\
    -v ./run_application_files/get_started.py:/get_started.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 get_started:my_first_app

1b2654848ee889dc3295d5aa11a0cc715fe91f00d8c4994efaafb4ac9fb21ef4


So now you can try it in your browser, but here I use the `curl` utility - it returns "hello", just as I declared in the programme.

In [2]:
!curl localhost:8000

"hello"

**Don't forget** to stop the image when you've finished playing with the container.

In [4]:
!docker stop test_container

test_container


##